In [1]:
from waybacknews.searchapi import SearchApiClient
from datetime import datetime
import pandas as pd
import requests
from retrying import retry
import requests_cache
from tqdm import tqdm
import mediacloud.api
from newspaper import Article
# import unicodedata
import concurrent.futures
from utils.utils import split_into_chunks, fetch_snippet, sanitize_snippet, get_snippet_from_wayback_machine

import weaviate
import json
import os

In [2]:
OPENAI_APIKEY = os.environ['OPENAI_API_KEY']
client = weaviate.Client(
    url = "https://semantic-search-cluster-mp9jmm6o.weaviate.network",  # Replace with your endpoint
    auth_client_secret=weaviate.AuthApiKey(api_key="kEzuSpJyK8J7IDLLsmzoBlH8mOtkrfkCjIH6"),  # Replace w/ your Weaviate instance API key
    additional_headers = {
        "X-OpenAI-Api-Key": OPENAI_APIKEY  # Replace with your inference API key
    }
)

if client.schema.exists("Article"):
    client.schema.delete_class("Article")

In [3]:
class_obj = {
    "class": "Article",
    "vectorizer": "text2vec-openai",  # If set to "none" you must always provide vectors yourself. Could be any other "text2vec-*" also.
    "moduleConfig": {
        "text2vec-openai": {},
        "generative-openai": {}  # Ensure the `generative-openai` module is used for generative queries
    }
}

client.schema.create_class(class_obj)

In [6]:
# get news domains
collection_id = "38379429"
directory_api = mediacloud.api.DirectoryApi(auth_token='e85cce24da8b73eaa05329d258146c044ef055db')
api = SearchApiClient("mediacloud")

all_domains = []
# Pagination setup
offset = 0
limit = 100  # seems to have a 100 limit
more_pages = True

while more_pages:
    # Fetch sources with current offset
    sources_response = directory_api.source_list(collection_id=collection_id, limit=limit, offset=offset)
    sources = sources_response.get('results', [])
    domains = [source['homepage'] for source in sources]
    all_domains.extend(domains)
    
    # Update the offset
    offset += limit
    
    # Check if there are more pages to fetch
    more_pages = len(sources) == limit

# Cleaning up domains
cleaned_domains = [
    domain.replace('https://www.', '')
          .replace('http://www.', '')
          .replace('https://', '')
          .replace('http://', '')
          .replace('#spider', '')
          .replace('/#spider', '')
          .rstrip('/')
    for domain in all_domains
    if domain  # Ensure the domain is not None or empty
]
print(f"Number of sources: {len(cleaned_domains)}")
domains_df = pd.DataFrame(cleaned_domains, columns=['Domain'])
# Save the DataFrame to a CSV file
# domains_df.to_csv('domains.csv', index=False)

domain_chunks = list(split_into_chunks(cleaned_domains, 1000))

# sources_response = directory_api.source_list(collection_id=collection_id)
# domains = [source['homepage'] for source in sources_response['results']]
# num_sources = len(sources_response['results'])
# print(f"Number of sources: {num_sources}")

# cleaned_domains = [domain.replace('https://www.', '').replace('http://www.', '').replace('https://','').replace('http://','') for domain in domains]
# cleaned_domains = [url.rstrip('/') for url in cleaned_domains]
# print(cleaned_domains)

# Enable requests cache
requests_cache.install_cache('article_cache', backend='filesystem', expire_after=3600)


# Query parameters
# query_term = '("police shooting" OR "shot by police" OR "police shot" OR "officer-involved shooting" OR "police-involved shooting" OR "police officer shooting" OR "police shot" OR "officer shot")'
query_term = '("police shooting" OR "shot by police" OR "police shot" OR "officer-involved shooting" OR "police-involved shooting" OR "police officer shooting" OR "officer shot" OR "deputy shot" OR "sheriff shot" OR "cop shot" OR "trooper shot" OR "shot by officer" OR "shot by deputy" OR "shot by sheriff" OR "shot by cop" OR "shot by trooper" OR \
    "killed by police" OR "killed by officer" OR "killed by deputy" OR "killed by sheriff" OR "killed by cop" OR "killed by trooper")'

# query_term = 'police AND shot'
start = datetime(2023, 12, 1) #11/6 - 11/15
end = datetime(2023, 12, 4) 
language = "en"

# DataFrame to store combined results
combined_results = pd.DataFrame()
results_list = []

for chunk in domain_chunks:
    domains_str = f"domain:({' OR '.join(chunk)})"
    query = f"{query_term} AND language:{language} AND {domains_str}"
    
    # Perform the search with the current chunk
    articles = []
    for list_of_articles in api.all_articles(query, start, end):
        articles.extend(list_of_articles)
        print(f"Found {len(articles)} articles")
    
    if articles:
        chunk_results = pd.DataFrame(articles)
        results_list.append(chunk_results)
    

# Concatenate all DataFrames in the list
combined_results = pd.concat(results_list, ignore_index=True)
# results = pd.DataFrame(articles).sort_values(by='publication_date', ascending=False)
combined_results.sort_values(by='publication_date', ascending=False, inplace=True)
print(combined_results.shape)
combined_results.drop_duplicates(subset=['title'], keep='first', inplace=True)
print("after dropping duplicates: ", combined_results.shape)
# combined_results.to_csv(f'./data_storage/{start}_no_snippet.csv', index=False)


for index, article in tqdm(combined_results.iterrows(), total=combined_results.shape[0]):
    # only use wayback machine
    wayback_url = article['article_url']
    snippet = get_snippet_from_wayback_machine(wayback_url)
    if snippet:
        sanitized_snippet = sanitize_snippet(snippet)
        combined_results.loc[index, 'snippet'] = sanitized_snippet
        
    # or concurrently run mediacloud wayback machine and newspaper3k
    # article_url = article['url']
    # wayback_url = article['article_url']
    # snippet, method_used = fetch_snippet(article_url, wayback_url)
    # if snippet:
    #     sanitized_snippet = sanitize_snippet(snippet)
    #     combined_results.loc[index, 'snippet'] = sanitized_snippet
    #     # print(f"Snippet fetched using {method_used} method for URL: {article_url}")

print(combined_results.shape)
combined_results.dropna(subset=['snippet'], inplace=True)
print("after dropping null snippets: ", combined_results.shape)
# combined_results.to_csv(f'./data_storage/{start}.csv', index=False)
print(f"Data retrieval complete. Results saved to './data_storage/{start}.csv'.")



Number of sources: 8913
Found 145 articles
Found 107 articles
Found 20 articles
Found 6 articles
Found 9 articles
Found 6 articles
Found 1 articles
Found 0 articles
Found 1 articles
(295, 9)
after dropping duplicates:  (148, 9)


100%|██████████| 148/148 [01:20<00:00,  1.85it/s]

(148, 10)
after dropping null snippets:  (148, 10)
Data retrieval complete. Results saved to './data_storage/2023-12-01 00:00:00.csv'.


In [8]:
with client.batch(batch_size=100) as batch:
    for index, row in combined_results.iterrows():
        properties = {
            "title": row['title'],
            "publication_date": row['publication_date'],
            "snippet": row['snippet']
        }
        batch.add_data_object(properties, "Article")
        print(f"Importing article: {index + 1}")


Importing article: 1
Importing article: 118
Importing article: 120
Importing article: 122
Importing article: 132
Importing article: 133
Importing article: 138
Importing article: 140
Importing article: 272
Importing article: 150
Importing article: 151
Importing article: 156
Importing article: 158
Importing article: 160
Importing article: 161
Importing article: 276
Importing article: 277
Importing article: 116
Importing article: 78
Importing article: 80
Importing article: 83
Importing article: 281
Importing article: 90
Importing article: 92
Importing article: 100
Importing article: 278
Importing article: 114
Importing article: 163
Importing article: 269
Importing article: 209
Importing article: 215
Importing article: 263
Importing article: 223
Importing article: 226
Importing article: 227
Importing article: 230
Importing article: 240
Importing article: 245
Importing article: 252
Importing article: 254
Importing article: 204
Importing article: 185
Importing article: 176
Importing article:

In [16]:
# nearText = {"concepts": ["police shooting, shot by police, police shot, officer-involved shooting, police-involved shooting, police officer shooting, officer shot, deputy shot, sheriff shot, cop shot, trooper shot, shot by officer, shot by deputy, shot by sheriff, shot by cop, shot by trooper, killed by police, killed by officer, killed by deputy, killed by sheriff, killed by cop, killed by trooper"]}
nearText = {"concepts": ["a recent fatal police shooting. The incident is about police officer fired shots and killed someone. The victims are dead."]}

response = (
    client.query
    .get("Article", ["title", "publication_date", "snippet"])
    .with_near_text(nearText)
    .with_generate(single_prompt="Summarize {snippet} in one sentence.")
    .with_limit(20)
    .do()
)

print(json.dumps(response, indent=4))
# convert response to dataframe and save to csv
df = pd.DataFrame(response['data']['Get']['Article'])
df.to_csv(f'../data_storage/{start}_weaviate.csv', index=False)

{
    "data": {
        "Get": {
            "Article": [
                {
                    "_additional": {
                        "generate": {
                            "error": null,
                            "singleResult": "A man was shot and killed by police in southeast Wichita after he pointed a gun at officers during a domestic dispute call."
                        }
                    },
                    "publication_date": "2023-12-03",
                    "snippet": "Officer-involved shooting in southeast Wichita leaves one man dead WICHITA, Kan. (KAKE) - One man has died after an officer-involved shooting in southeast Wichita on Sunday. Wichita Police says officers were dispatched to the 4200 block of East Gilbert St. just after 2:15 p.m. for the report of a domestic dispute. Upon arrival, officers found the calling party, an adult female, who informed police the suspect was in the backyard making suicidal statements. According to police, officers made conta

------
Not used

In [2]:
collection_id = "38379429"
directory_api = mediacloud.api.DirectoryApi(auth_token='e85cce24da8b73eaa05329d258146c044ef055db')
api = SearchApiClient("mediacloud")

In [17]:
# Function to split the domain list into chunks
def split_into_chunks(domains, chunk_size):
    for i in range(0, len(domains), chunk_size):
        yield domains[i:i + chunk_size]


In [4]:
all_domains = []
# Pagination setup
offset = 0
limit = 100  # seems to have a 100 limit
more_pages = True

while more_pages:
    # Fetch sources with current offset
    sources_response = directory_api.source_list(collection_id=collection_id, limit=limit, offset=offset)
    sources = sources_response.get('results', [])
    domains = [source['homepage'] for source in sources]
    all_domains.extend(domains)
    
    # Update the offset
    offset += limit
    
    # Check if there are more pages to fetch
    more_pages = len(sources) == limit

# Cleaning up domains
cleaned_domains = [
    domain.replace('https://www.', '').replace('http://www.', '').replace('https://','').replace('http://','').rstrip('/')
    for domain in all_domains
    if domain  # Ensure the domain is not None or empty
]
print(f"Number of sources: {len(cleaned_domains)}")

domains_df = pd.DataFrame(cleaned_domains, columns=['Domain'])
# Save the DataFrame to a CSV file
domains_df.to_csv('domains.csv', index=False)

Number of sources: 8913


In [21]:
domain_chunks = list(split_into_chunks(cleaned_domains, 1000))

# sources_response = directory_api.source_list(collection_id=collection_id)
# domains = [source['homepage'] for source in sources_response['results']]
# num_sources = len(sources_response['results'])
# print(f"Number of sources: {num_sources}")

# cleaned_domains = [domain.replace('https://www.', '').replace('http://www.', '').replace('https://','').replace('http://','') for domain in domains]
# cleaned_domains = [url.rstrip('/') for url in cleaned_domains]
# print(cleaned_domains)

# Enable requests cache
requests_cache.install_cache('article_cache', backend='filesystem', expire_after=3600)


# Query parameters
query_term = '("police shooting" OR "shot by police" OR "police shot" OR "officer-involved shooting" OR "police-involved shooting" OR "police officer shooting" OR "police shot" OR "officer shot")'
# query_term = '("police shooting" OR "shot by police" OR "police shot" OR "officer-involved shooting" OR "police-involved shooting" OR "police officer shooting" OR "police shot" OR "officer shot" OR "deputy shot" OR "sheriff shot" OR "cop shot")'
# query_term = 'police AND shot'
start = datetime(2023, 9, 1) #11/6 - 11/15
end = datetime(2023, 10, 1) 
language = "en"

# DataFrame to store combined results
combined_results = pd.DataFrame()
results_list = []

for chunk in domain_chunks:
    domains_str = f"domain:({' OR '.join(chunk)})"
    query = f"{query_term} AND language:{language} AND {domains_str}"
    
    # Perform the search with the current chunk
    articles = []
    for list_of_articles in api.all_articles(query, start, end):
        articles.extend(list_of_articles)
        print(f"all_articles endpoint: {len(articles)} articles")
    
    if articles:
        chunk_results = pd.DataFrame(articles)
        results_list.append(chunk_results)
    

# Concatenate all DataFrames in the list
combined_results = pd.concat(results_list, ignore_index=True)
combined_results.sort_values(by='publication_date', ascending=False, inplace=True)
print(combined_results.head())


all_articles endpoint: 622 articles
all_articles endpoint: 261 articles
all_articles endpoint: 67 articles
all_articles endpoint: 15 articles
all_articles endpoint: 34 articles
all_articles endpoint: 25 articles
all_articles endpoint: 14 articles
all_articles endpoint: 14 articles
all_articles endpoint: 2 articles
                                                 title publication_date  \
256  Antioch: Man wanted for homicide is shot by po...       2023-10-01   
112  Antioch: Man wanted for homicide is shot by po...       2023-10-01   
328  1 killed in police shooting at Pa. traffic sto...       2023-10-01   
417  St. Louis police shoot man who threatened rela...       2023-10-01   
173  Demonstrators call for justice after deadly po...       2023-10-01   

             capture_time language            domain  \
256  2023-10-03T11:47:16Z       en   mercurynews.com   
112  2023-10-03T13:10:17Z       en  eastbaytimes.com   
328  2023-10-13T01:29:15Z       en      pennlive.com   
417  2023

In [22]:
combined_results.shape

(1054, 9)

In [23]:
def get_snippet_from_newspaper3k(url):
    extracted_article = Article(url)
    extracted_article.download()
    extracted_article.parse()
    return extracted_article.text

def get_snippet_from_wayback_machine(url):
    response = requests.get(url)
    response.raise_for_status()
    snippet = response.json().get('snippet', '')
    # return response.text 
    return snippet

def fetch_snippet(article_url, wayback_url):
    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        future_to_method = {
            executor.submit(get_snippet_from_newspaper3k, article_url): 'newspaper3k',
            executor.submit(get_snippet_from_wayback_machine, wayback_url): 'wayback'
        }
        for future in concurrent.futures.as_completed(future_to_method):
            method = future_to_method[future]
            try:
                data = future.result()
                return data, method
            except Exception as exc:
                print(f"{method} method failed with exception: {exc}")


def sanitize_snippet(snippet):
    sanitized_snippet = snippet.replace('\n', ' ').replace('\r', '').strip()
    # Normalize Unicode characters
    # normalized_snippet = unicodedata.normalize('NFKD', sanitized_snippet)
    return sanitized_snippet

In [24]:
for index, article in tqdm(combined_results.iterrows(), total=combined_results.shape[0]):
    article_url = article['url']
    wayback_url = article['article_url']
    snippet, method_used = fetch_snippet(article_url, wayback_url)
    if snippet:
        sanitized_snippet = sanitize_snippet(snippet)
        combined_results.loc[index, 'snippet'] = sanitized_snippet
        print(f"Snippet fetched using {method_used} method for URL: {article_url}")

  0%|          | 1/1054 [00:09<2:38:28,  9.03s/it]

Snippet fetched using newspaper3k method for URL: https://www.mercurynews.com/2023/10/01/antioch-man-wanted-for-homicide-earlier-this-year-shot-by-police-during-pursuit/


  0%|          | 2/1054 [00:25<3:59:23, 13.65s/it]

Snippet fetched using newspaper3k method for URL: https://www.eastbaytimes.com/2023/10/01/antioch-man-wanted-for-homicide-earlier-this-year-shot-by-police-during-pursuit/


  0%|          | 3/1054 [00:26<2:16:54,  7.82s/it]

...

Snippet fetched using wayback method for URL: https://www.sandiegouniontribune.com/news/nation-world/story/2023-09-29/2-mexican-migrants-shot-dead-3-injured-in-dawn-attack-on-us-border-near-tecate-mexico


  6%|▌         | 64/1054 [04:39<16:56,  1.03s/it]

Snippet fetched using newspaper3k method for URL: https://nypost.com/2023/09/29/influencer-meatball-regrets-part-in-riots-family-ashamed/


  6%|▌         | 64/1054 [04:40<1:12:20,  4.38s/it]


KeyboardInterrupt: 

In [ ]:
combined_results.to_csv(f'./data_storage/{start}.csv', index=False)
print(f"Data retrieval complete. Results saved to './data_storage/{start}.csv'.")